# Establish Prediction Model for Nb Consultation on Doctolib

### Using rappel_conso_viandes.csv to create a prediction model for nb. consultation
rappel_conso_viandes.csv is created from web scraping using getNews.py

In [1]:
import pandas as pd

df_rappel = pd.read_csv("rappel_conso_viandes.csv")
df_CS = pd.read_csv("total_CS.csv")

In [2]:
df_rappel.head(5)

,title,maker,risks,reason,date,zone,link
0,Poitrine fumée,sans marque (vendu au rayon stand avec service),Risques :Salmonella spp (agent responsable de ...,Motif :Présence de Salmonelles,29/12/2023 16:23:03,cora Evreux (27),https://rappel.conso.gouv.fr/fiche-rappel/1336...
1,Poitrine fumée,sans marque (vendu au rayon libre service emba...,Risques :Salmonella spp (agent responsable de ...,Motif :Présence de Salmonelles,29/12/2023 16:21:47,cora Evreux (27),https://rappel.conso.gouv.fr/fiche-rappel/1336...
2,viande Hachée,viande hachée à la demande,Risques :Listeria monocytogenes (agent respons...,Motif :Présence de Listeria,29/12/2023 15:49:20,Wormhout + 10 kms autour,https://rappel.conso.gouv.fr/fiche-rappel/1337...
3,SAUCISSES DE VOLAILLE X6 300G et SAUCISSES VOL...,VOLANDRIE,"Risques :Inertes (verre, métal, plastique, pap...",Motif :Risque de présence de plastique dur tra...,29/12/2023 09:17:58,France entière,https://rappel.conso.gouv.fr/fiche-rappel/1336...
4,Poitrine fumée,sans marque (vendu au rayon stand avec service...,Risques :Salmonella spp (agent responsable de ...,Motif :Présence de Salmonelles,29/12/2023 08:46:31,cora Evreux (27),https://rappel.conso.gouv.fr/fiche-rappel/1335...


In [3]:
df_CS.head(5)

,region,total_CS,year
0,Auvergne-Rhones-Alpes,16012537,2021
1,Auvergne-Rhones-Alpes,18821994,2022
2,Auvergne-Rhones-Alpes,21006355,2023
3,Bourgogne-Franche-Comte,4229363,2021
4,Bourgogne-Franche-Comte,4870902,2022


In [14]:
import pandas as pd
import re
import unidecode

departement_to_region = {
    # Auvergne-Rhône-Alpes
    "Ain": "Auvergne-Rhône-Alpes", "01": "Auvergne-Rhône-Alpes",
    "Allier": "Auvergne-Rhône-Alpes", "03": "Auvergne-Rhône-Alpes",
    "Ardèche": "Auvergne-Rhône-Alpes", "07": "Auvergne-Rhône-Alpes",
    "Cantal": "Auvergne-Rhône-Alpes", "15": "Auvergne-Rhône-Alpes",
    "Drôme": "Auvergne-Rhône-Alpes", "26": "Auvergne-Rhône-Alpes",
    "Isère": "Auvergne-Rhône-Alpes", "38": "Auvergne-Rhône-Alpes",
    "Loire": "Auvergne-Rhône-Alpes", "42": "Auvergne-Rhône-Alpes",
    "Haute-Loire": "Auvergne-Rhône-Alpes", "43": "Auvergne-Rhône-Alpes",
    "Puy-de-Dôme": "Auvergne-Rhône-Alpes", "63": "Auvergne-Rhône-Alpes",
    "Rhône": "Auvergne-Rhône-Alpes", "69": "Auvergne-Rhône-Alpes",
    "Savoie": "Auvergne-Rhône-Alpes", "73": "Auvergne-Rhône-Alpes",
    "Haute-Savoie": "Auvergne-Rhône-Alpes", "74": "Auvergne-Rhône-Alpes",

    # Bourgogne-Franche-Comté
    "Côte-d'Or": "Bourgogne-Franche-Comté", "21": "Bourgogne-Franche-Comté",
    "Doubs": "Bourgogne-Franche-Comté", "25": "Bourgogne-Franche-Comté",
    "Jura": "Bourgogne-Franche-Comté", "39": "Bourgogne-Franche-Comté",
    "Nièvre": "Bourgogne-Franche-Comté", "58": "Bourgogne-Franche-Comté",
    "Haute-Saône": "Bourgogne-Franche-Comté", "70": "Bourgogne-Franche-Comté",
    "Saône-et-Loire": "Bourgogne-Franche-Comté", "71": "Bourgogne-Franche-Comté",
    "Yonne": "Bourgogne-Franche-Comté", "89": "Bourgogne-Franche-Comté",
    "Territoire de Belfort": "Bourgogne-Franche-Comté", "90": "Bourgogne-Franche-Comté",

    # Bretagne
    "Côtes-d'Armor": "Bretagne", "22": "Bretagne",
    "Finistère": "Bretagne", "29": "Bretagne",
    "Ille-et-Vilaine": "Bretagne", "35": "Bretagne",
    "Morbihan": "Bretagne", "56": "Bretagne",

    # Centre-Val de Loire
    "Cher": "Centre-Val de Loire", "18": "Centre-Val de Loire",
    "Eure-et-Loir": "Centre-Val de Loire", "28": "Centre-Val de Loire",
    "Indre": "Centre-Val de Loire", "36": "Centre-Val de Loire",
    "Indre-et-Loire": "Centre-Val de Loire", "37": "Centre-Val de Loire",
    "Loir-et-Cher": "Centre-Val de Loire", "41": "Centre-Val de Loire",
    "Loiret": "Centre-Val de Loire", "45": "Centre-Val de Loire",

    # Corse
    "Corse-du-Sud": "Corse", "2A": "Corse",
    "Haute-Corse": "Corse", "2B": "Corse",

    # Grand Est
    "Ardennes": "Grand Est", "08": "Grand Est",
    "Aube": "Grand Est", "10": "Grand Est",
    "Marne": "Grand Est", "51": "Grand Est",
    "Haute-Marne": "Grand Est", "52": "Grand Est",
    "Meurthe-et-Moselle": "Grand Est", "54": "Grand Est",
    "Meuse": "Grand Est", "55": "Grand Est",
    "Moselle": "Grand Est", "57": "Grand Est",
    "Bas-Rhin": "Grand Est", "67": "Grand Est",
    "Haut-Rhin": "Grand Est", "68": "Grand Est",
    "Vosges": "Grand Est", "88": "Grand Est",

    # Hauts-de-France
    "Aisne": "Hauts-de-France", "02": "Hauts-de-France",
    "Nord": "Hauts-de-France", "59": "Hauts-de-France",
    "Oise": "Hauts-de-France", "60": "Hauts-de-France",
    "Pas-de-Calais": "Hauts-de-France", "62": "Hauts-de-France",
    "Somme": "Hauts-de-France", "80": "Hauts-de-France",

    # Île-de-France
    "Paris": "Île-de-France", "75": "Île-de-France",
    "Seine-et-Marne": "Île-de-France", "77": "Île-de-France",
    "Yvelines": "Île-de-France", "78": "Île-de-France",
    "Essonne": "Île-de-France", "91": "Île-de-France",
    "Hauts-de-Seine": "Île-de-France", "92": "Île-de-France",
    "Seine-Saint-Denis": "Île-de-France", "93": "Île-de-France",
    "Val-de-Marne": "Île-de-France", "94": "Île-de-France",
    "Val-d'Oise": "Île-de-France", "95": "Île-de-France",

    # Normandie
    "Calvados": "Normandie", "14": "Normandie",
    "Eure": "Normandie", "27": "Normandie",
    "Manche": "Normandie", "50": "Normandie",
    "Orne": "Normandie", "61": "Normandie",
    "Seine-Maritime": "Normandie", "76": "Normandie",

    # Nouvelle-Aquitaine
    "Charente": "Nouvelle-Aquitaine", "16": "Nouvelle-Aquitaine",
    "Charente-Maritime": "Nouvelle-Aquitaine", "17": "Nouvelle-Aquitaine",
    "Corrèze": "Nouvelle-Aquitaine", "19": "Nouvelle-Aquitaine",
    "Creuse": "Nouvelle-Aquitaine", "23": "Nouvelle-Aquitaine",
    "Dordogne": "Nouvelle-Aquitaine", "24": "Nouvelle-Aquitaine",
    "Gironde": "Nouvelle-Aquitaine", "33": "Nouvelle-Aquitaine",
    "Landes": "Nouvelle-Aquitaine", "40": "Nouvelle-Aquitaine",
    "Lot-et-Garonne": "Nouvelle-Aquitaine", "47": "Nouvelle-Aquitaine",
    "Pyrénées-Atlantiques": "Nouvelle-Aquitaine", "64": "Nouvelle-Aquitaine",
    "Deux-Sèvres": "Nouvelle-Aquitaine", "79": "Nouvelle-Aquitaine",
    "Vienne": "Nouvelle-Aquitaine", "86": "Nouvelle-Aquitaine",
    "Haute-Vienne": "Nouvelle-Aquitaine", "87": "Nouvelle-Aquitaine",

    # Occitanie
    "Ariège": "Occitanie", "09": "Occitanie",
    "Aude": "Occitanie", "11": "Occitanie",
    "Aveyron": "Occitanie", "12": "Occitanie",
    "Gard": "Occitanie", "30": "Occitanie",
    "Haute-Garonne": "Occitanie", "31": "Occitanie",
    "Gers": "Occitanie", "32": "Occitanie",
    "Hérault": "Occitanie", "34": "Occitanie",
    "Lot": "Occitanie", "46": "Occitanie",
    "Lozère": "Occitanie", "48": "Occitanie",
    "Hautes-Pyrénées": "Occitanie", "65": "Occitanie",
    "Pyrénées-Orientales": "Occitanie", "66": "Occitanie",
    "Tarn": "Occitanie", "81": "Occitanie",
    "Tarn-et-Garonne": "Occitanie", "82": "Occitanie",

    # Pays de la Loire
    "Loire-Atlantique": "Pays de la Loire", "44": "Pays de la Loire",
    "Maine-et-Loire": "Pays de la Loire", "49": "Pays de la Loire",
    "Mayenne": "Pays de la Loire", "53": "Pays de la Loire",
    "Sarthe": "Pays de la Loire", "72": "Pays de la Loire",
    "Vendée": "Pays de la Loire", "85": "Pays de la Loire",

    # Provence-Alpes-Côte d'Azur
    "Alpes-de-Haute-Provence": "Provence-Alpes-Côte d'Azur", "04": "Provence-Alpes-Côte d'Azur",
    "Hautes-Alpes": "Provence-Alpes-Côte d'Azur", "05": "Provence-Alpes-Côte d'Azur",
    "Alpes-Maritimes": "Provence-Alpes-Côte d'Azur", "06": "Provence-Alpes-Côte d'Azur",
    "Bouches-du-Rhône": "Provence-Alpes-Côte d'Azur", "13": "Provence-Alpes-Côte d'Azur",
    "Var": "Provence-Alpes-Côte d'Azur", "83": "Provence-Alpes-Côte d'Azur",
    "Vaucluse": "Provence-Alpes-Côte d'Azur", "84": "Provence-Alpes-Côte d'Azur",
}


# 处理 zone 列，将 département 提取并映射到 région
def extract_regions(zone):
    if pd.isna(zone) or not isinstance(zone, str):
        return ["Unknown"]

    # 统一大小写 & 去除重音
    zone = unidecode.unidecode(zone).strip()

    # 处理 "France entière"
    if "FRANCE ENTIERE" in zone.upper() or "FRANCE" in zone.upper():
        return ["All Regions"]

    # 处理 "Régions : XXX, XXX"
    if "REGIONS :" in zone.upper():
        regions = re.findall(r"([A-Za-zÀ-ÿ-']+)", zone)
        matched_regions = [r for r in regions if r in departement_to_region.values()]
        return list(set(matched_regions)) if matched_regions else ["Unknown"]

    # 处理 "Départements : XXX (XX), XXX (XX)"
    if "DEPARTEMENTS :" in zone.upper():
        departements = re.findall(r"\b([A-Za-zÀ-ÿ-']+|\d{2,3})\b", zone)  # 获取 département 名称或数字编号
        matched_regions = list(set(departement_to_region.get(dep, "Unknown") for dep in departements))
        return matched_regions if matched_regions else ["Unknown"]

    # 处理 "city (DÉPARTEMENT编号)"
    match = re.search(r"\((\d{2,3}|2A|2B)\)", zone)  # 提取括号中的 département 编号
    if match:
        departement_code = match.group(1)
        return [departement_to_region.get(departement_code, "Unknown")]

    return ["Unknown"]

# 读取数据
df_rappel = pd.read_csv("rappel_conso_viandes.csv", sep=",")  # 确保路径正确

# 应用转换
df_rappel["region"] = df_rappel["zone"].apply(extract_regions)

# 展示前几行
print(df_rappel[["zone", "region"]].head(10))  # 显示前 10 行数据

# 统计未匹配的 zone
unmatched_zones = df_rappel[df_rappel["region"].apply(lambda x: "Unknown" in x)]
print(f"未匹配的 zone 数量: {len(unmatched_zones)}")

# 如果有未匹配的值，保存以便手动检查
if not unmatched_zones.empty:
    unmatched_zones.to_csv("unmatched_zones.csv", index=False)
    print("未匹配的 zone 已保存到 unmatched_zones.csv，请检查！")


                       zone         region
0          cora Evreux (27)    [Normandie]
1          cora Evreux (27)    [Normandie]
2  Wormhout + 10 kms autour      [Unknown]
3            France entière  [All Regions]
4          cora Evreux (27)    [Normandie]
5            France entière  [All Regions]
6            France entière  [All Regions]
7            France entière  [All Regions]
8  61110 Moutiers au Perche      [Unknown]
9            France entière  [All Regions]
未匹配的 zone 数量: 471
未匹配的 zone 已保存到 unmatched_zones.csv，请检查！


In [16]:
import os
import re
import pandas as pd
import unidecode
from dotenv import load_dotenv
from mistralai import Mistral

# ---------------------------
# 加载环境变量并设置 Mistral API 客户端
# ---------------------------
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

client = Mistral(api_key=MISTRAL_API_KEY)
model = "mistral-large-latest"

def chat_with_mistral(user_input: str) -> str:
    """调用 Mistral AI 的 API 并返回响应内容。"""
    try:
        response = client.chat.complete(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": user_input},
            ],
            tool_choice="any",
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# ---------------------------
# 部门到地区的映射字典（示例，需补充完整）
# ---------------------------
departement_to_region = {
    # Auvergne-Rhone-Alpes 示例
    "Ain": "Auvergne-Rhone-Alpes", "01": "Auvergne-Rhone-Alpes",
    "Allier": "Auvergne-Rhone-Alpes", "03": "Auvergne-Rhone-Alpes",
    # 其他 mapping ...
}

# ---------------------------
# 通过 zone 字段提取地区信息
# ---------------------------
def extract_regions(zone):
    """根据 zone 字段的描述，通过正则和映射字典提取地区信息。"""
    if pd.isna(zone) or not isinstance(zone, str):
        return ["Unknown"]
    
    # 统一大小写并去除重音
    zone = unidecode.unidecode(zone).strip()

    # 处理 "France entière" 情况
    if "FRANCE ENTIERE" in zone.upper() or "FRANCE" in zone.upper():
        return ["All Regions"]
    
    # 处理 "REGIONS : XXX, XXX"
    if "REGIONS :" in zone.upper():
        regions = re.findall(r"([A-Za-zÀ-ÿ-']+)", zone)
        matched_regions = [r for r in regions if r in departement_to_region.values()]
        return list(set(matched_regions)) if matched_regions else ["Unknown"]
    
    # 处理 "DEPARTEMENTS : XXX (XX), XXX (XX)"
    if "DEPARTEMENTS :" in zone.upper():
        departements = re.findall(r"\b([A-Za-zÀ-ÿ-']+|\d{2,3})\b", zone)
        matched_regions = list(set(departement_to_region.get(dep, "Unknown") for dep in departements))
        return matched_regions if matched_regions else ["Unknown"]
    
    # 处理 "city (DÉPARTEMENT编号)" 情况
    match = re.search(r"\((\d{2,3}|2A|2B)\)", zone)
    if match:
        departement_code = match.group(1)
        return [departement_to_region.get(departement_code, "Unknown")]
    
    return ["Unknown"]

# ---------------------------
# 读取 CSV 数据，并初步提取 region
# ---------------------------
df_rappel = pd.read_csv("rappel_conso_viandes.csv", sep=",")
df_rappel["region"] = df_rappel["zone"].apply(extract_regions)

# 展示前 10 行数据
print(df_rappel[["zone", "region"]].head(10))

# 找出 region 为 Unknown 的记录
unmatched_zones = df_rappel[df_rappel["region"].apply(lambda x: "Unknown" in x)]
print(f"未匹配的 zone 数量: {len(unmatched_zones)}")
if not unmatched_zones.empty:
    unmatched_zones.to_csv("unmatched_zones.csv", index=False)
    print("未匹配的 zone 已保存到 unmatched_zones.csv，请检查！")

# ---------------------------
# 调用 LLM 补全未匹配的 region
# ---------------------------
def get_region_from_zone_with_llm(zone: str) -> str:
    """
    根据 zone 描述构造 prompt，通过 Mistral LLM 获取建议的法国地区名称。
    """
    prompt = (
        f"请根据以下描述确定法国所属的地区：\n\n"
        f"'{zone}'\n\n"
        "请仅返回对应的法国地区名称（例如：Auvergne-Rhone-Alpes, Ile-de-France, "
        "Provence-Alpes-Cote d'Azur 等），如果无法确定，请返回'Unknown'。"
    )
    response = chat_with_mistral(prompt)
    return response.strip()

# 遍历未匹配的 zone，调用 LLM 获取建议的 region 并更新 DataFrame
for idx, row in unmatched_zones.iterrows():
    zone_text = row["zone"]
    suggested_region = get_region_from_zone_with_llm(zone_text)
    print(f"Zone: {zone_text} -> 建议的 Region: {suggested_region}")
    # 更新 region 字段（这里保持为列表形式存储，如有需要可自行转换）
    df_rappel.at[idx, "region"] = [suggested_region]

# 保存更新后的数据
df_rappel.to_csv("rappel_conso_viandes_updated.csv", index=False)
print("更新后的数据已保存到 rappel_conso_viandes_updated.csv")


                       zone         region
0          cora Evreux (27)      [Unknown]
1          cora Evreux (27)      [Unknown]
2  Wormhout + 10 kms autour      [Unknown]
3            France entière  [All Regions]
4          cora Evreux (27)      [Unknown]
5            France entière  [All Regions]
6            France entière  [All Regions]
7            France entière  [All Regions]
8  61110 Moutiers au Perche      [Unknown]
9            France entière  [All Regions]
未匹配的 zone 数量: 512
未匹配的 zone 已保存到 unmatched_zones.csv，请检查！
Zone: cora Evreux (27) -> 建议的 Region: Normandie
Zone: cora Evreux (27) -> 建议的 Region: Normandie
Zone: Wormhout + 10 kms autour -> 建议的 Region: Hauts-de-France
Zone: cora Evreux (27) -> 建议的 Region: Normandie
Zone: 61110 Moutiers au Perche -> 建议的 Region: Normandie
Zone: Départements : 

BOUCHES-DU-RHONE (13), RHONE (69) -> 建议的 Region: Provence-Alpes-Cote d'Azur
Zone: Départements : 

PUY-DE-DOME (63) -> 建议的 Region: Auvergne-Rhone-Alpes
Zone: Villebois Lavalette -> 建议的

In [23]:
import os
import re
import ast
import pandas as pd
import unidecode
from dotenv import load_dotenv
from tqdm import tqdm


# ---------------------------
# 1. Load environment variables and set up Mistral API client
# ---------------------------
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

client = Mistral(api_key=MISTRAL_API_KEY)
model = "mistral-large-latest"

def chat_with_mistral(user_input: str) -> str:
    """Call Mistral AI's API and return the response."""
    try:
        response = client.chat.complete(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": user_input},
            ],
            tool_choice="any",
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# ---------------------------
# 2. Define the 13 canonical region names
# ---------------------------
canonical_regions = [
    "Auvergne-Rhones-Alpes",
    "Bourgogne-Franche-Comte",
    "Bretagne",
    "Centre-Val-De-Loire",
    "Corse",
    "Grand-Est",
    "Hauts-De-France",
    "Ile-De-France",
    "Normandie",
    "Nouvelle-Aquitaine",
    "Occitanie",
    "Pays-De-La-Loire",
    "Provence-Alpes-Cote-D-Azur"
]

# ---------------------------
# 3. Define a function that maps a given text to a list of canonical regions using the LLM
# ---------------------------
def llm_extract_region_list(zone_text: str) -> list:
    """
    Given a text description (zone_text), call the Mistral LLM to return a list of canonical region names
    (from the 13 defined above) that match the description. The list may contain multiple regions or be empty.
    """
    prompt = f"""
We have the following text describing one or more regions in France:
"{zone_text}"

Please select from the list of 13 canonical region names below those that match the description, and return them as a Python list:
{canonical_regions}

If you cannot determine a match or there are no matching regions, please return an empty list [].
Only provide a Python list without any additional explanation.
"""
    response = chat_with_mistral(prompt).strip()
    
    try:
        region_list = ast.literal_eval(response)
        if not isinstance(region_list, list):
            return []
    except Exception:
        # If literal_eval fails, try splitting by comma
        region_list = [r.strip() for r in response.split(",")]

    # Filter out any values not in canonical_regions and remove duplicates
    region_list = list({r for r in region_list if r in canonical_regions})
    return region_list

# ---------------------------
# 4. Read the original CSV and standardize the region column using the LLM
# ---------------------------
df_rappel = pd.read_csv("rappel_conso_viandes.csv", sep=",")

# Enable the progress bar for the apply function
tqdm.pandas()

# Process the 'zone' column row-by-row with a progress bar
df_rappel["region"] = df_rappel["zone"].progress_apply(llm_extract_region_list)

# ---------------------------
# 5. Check and export the results
# ---------------------------
# Display the first 10 rows to inspect the results
print(df_rappel[["zone", "region"]].head(10))

# Explode the 'region' column to count the occurrences of each region
df_exploded = df_rappel.explode("region")
region_counts = df_exploded["region"].value_counts()
print("\nOccurrences of each region:")
print(region_counts)

# Save the updated DataFrame to a new CSV file
df_rappel.to_csv("rappel_conso_viandes_updated.csv", index=False)
print("Saved to rappel_conso_viandes_updated.csv")

100%|██████████| 1456/1456 [43:07<00:00,  1.78s/it] 

                       zone                                             region
0          cora Evreux (27)                                                 []
1          cora Evreux (27)                                        [Normandie]
2  Wormhout + 10 kms autour                                                 []
3            France entière  [Bourgogne-Franche-Comte, Corse, Occitanie, Ha...
4          cora Evreux (27)                                        [Normandie]
5            France entière  [Bourgogne-Franche-Comte, Corse, Occitanie, Ha...
6            France entière  [Bourgogne-Franche-Comte, Corse, Occitanie, Ha...
7            France entière  [Bourgogne-Franche-Comte, Corse, Occitanie, Ha...
8  61110 Moutiers au Perche                                        [Normandie]
9            France entière  [Bourgogne-Franche-Comte, Corse, Occitanie, Ha...

Occurrences of each region:
region
Grand-Est                     959
Occitanie                     933
Bourgogne-Franche-Comte    

In [26]:
# Parse the 'date' column (which has a day-first format plus time)
df_rappel["date_parsed"] = pd.to_datetime(df_rappel["date"], dayfirst=True)

# Extract the year from the parsed date
df_rappel["year"] = df_rappel["date_parsed"].dt.year

# (Optional) If you no longer need the parsed date column, you can drop it
# df_rappel.drop(columns=["date_parsed"], inplace=True)

In [31]:
# Explode the 'region' column if it is a list (so each region becomes its own row)
df_exploded = df_rappel.explode("region")

# Group by region and year, counting how many rows (mentions) each pair has
df_count = df_exploded.groupby(["region", "year"]).size().reset_index(name="count_risk")

# Now df_count has columns: ['region', 'year', 'count_risk']

In [32]:
print(df_count.head(10))

                    region  year  count_risk
0    Auvergne-Rhones-Alpes  2021         250
1    Auvergne-Rhones-Alpes  2022         319
2    Auvergne-Rhones-Alpes  2023         336
3  Bourgogne-Franche-Comte  2021         255
4  Bourgogne-Franche-Comte  2022         324
5  Bourgogne-Franche-Comte  2023         346
6                 Bretagne  2021         255
7                 Bretagne  2022         323
8                 Bretagne  2023         322
9      Centre-Val-De-Loire  2021         249


In [40]:
# 1. Group by both region and year so each region-year pair has a unique total_CS
df_CS_unique = df_CS.groupby(["region", "year"], as_index=False)["total_CS"].sum()

# 2. Merge on both region and year
df_merged = df_count.merge(df_CS_unique, on=["region", "year"], how="left")

In [41]:
print(df_merged.head(10))

                    region  year  count_risk  total_CS
0    Auvergne-Rhones-Alpes  2021         250  16012537
1    Auvergne-Rhones-Alpes  2022         319  18821994
2    Auvergne-Rhones-Alpes  2023         336  21006355
3  Bourgogne-Franche-Comte  2021         255   4229363
4  Bourgogne-Franche-Comte  2022         324   4870902
5  Bourgogne-Franche-Comte  2023         346   5543627
6                 Bretagne  2021         255   6193303
7                 Bretagne  2022         323   7328236
8                 Bretagne  2023         322   8317998
9      Centre-Val-De-Loire  2021         249   3277933


In [43]:
# Save the updated DataFrame to a new CSV file
df_merged.to_csv("CS_rappel_viandes.csv", index=False)
print("Saved to CS_rappel_viandes.csv")

Saved to CS_rappel_viandes.csv


## Add Urban Heat Island Intensitty and CO2 Emission to the csv

In [46]:
df_model = pd.read_csv("CS_env.csv")
print(df_model.head(10))

                    region  year  count_risk  total_CS  UHII    CO2
0    Auvergne-Rhones-Alpes  2021         250  16012537   2.5  50000
1    Auvergne-Rhones-Alpes  2022         319  18821994   2.6  52000
2    Auvergne-Rhones-Alpes  2023         336  21006355   2.7  54000
3  Bourgogne-Franche-Comte  2021         255   4229363   2.0  30000
4  Bourgogne-Franche-Comte  2022         324   4870902   2.1  31000
5  Bourgogne-Franche-Comte  2023         346   5543627   2.2  32000
6                 Bretagne  2021         255   6193303   1.5  25000
7                 Bretagne  2022         323   7328236   1.5  25500
8                 Bretagne  2023         322   8317998   1.6  26000
9      Centre-Val-De-Loire  2021         249   3277933   1.8  20000


## Building our prediction model

In [60]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder

from tpot import TPOTRegressor

In [61]:
data = pd.read_csv("CS_env.csv")
data.head()

,region,year,count_risk,total_CS,UHII,CO2
0,Auvergne-Rhones-Alpes,2021,250,16012537,2.5,50000
1,Auvergne-Rhones-Alpes,2022,319,18821994,2.6,52000
2,Auvergne-Rhones-Alpes,2023,336,21006355,2.7,54000
3,Bourgogne-Franche-Comte,2021,255,4229363,2.0,30000
4,Bourgogne-Franche-Comte,2022,324,4870902,2.1,31000


In [62]:
# 查看缺失值情况
print(data.isnull().sum())

# 将region用LabelEncoder转换为数值
label_encoder = LabelEncoder()
data['region_encoded'] = label_encoder.fit_transform(data['region'])

# 简单查看数据处理结果
data.head()

region        0
year          0
count_risk    0
total_CS      0
UHII          0
CO2           0
dtype: int64


,region,year,count_risk,total_CS,UHII,CO2,region_encoded
0,Auvergne-Rhones-Alpes,2021,250,16012537,2.5,50000,0
1,Auvergne-Rhones-Alpes,2022,319,18821994,2.6,52000,0
2,Auvergne-Rhones-Alpes,2023,336,21006355,2.7,54000,0
3,Bourgogne-Franche-Comte,2021,255,4229363,2.0,30000,1
4,Bourgogne-Franche-Comte,2022,324,4870902,2.1,31000,1


In [63]:
# 定义特征X和目标y
# 这里示例使用 region_encoded, year, count_risk, UHII, CO2 作为特征
X = data[['region_encoded', 'year', 'count_risk', 'UHII', 'CO2']]
y = data['total_CS']

# 将数据拆分为训练集(80%)和测试集(20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

print("Size of training set:", X_train.shape, y_train.shape)
print("Size of testing set:", X_test.shape, y_test.shape)

Size of training set: (31, 5) (31,)
Size of testing set: (8, 5) (8,)


In [79]:
# 定义一个RepeatedKFold对象
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# 配置TPOT
tpot = TPOTRegressor(
    generations=5,           # 迭代次数，可根据实际需求增大
    population_size=20,      # 种群大小
    verbosity=2,             # 日志输出级别
    scoring='r2',            # 这里用R^2作为评估指标，也可用MAE/MSE
    random_state=42,         # 结果可复现
    n_jobs=-1,               # 并行加速，-1表示使用全部CPU
    cv=rkf                   # 将我们定义的RepeatedKFold传给TPOT
)

# 在训练集上进行自动化建模
tpot.fit(X_train, y_train)

is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor


/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/sklearn/base.py:1230: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(
/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/sklearn/base.py:1270: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(


is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Library/Fr

                                                                             
Generation 1 - Current best internal CV score: 0.8143257096062387
                                                                             
Generation 2 - Current best internal CV score: 0.8143679640935062
                                                                             
Generation 3 - Current best internal CV score: 0.8789325333939894
                                                                             
Generation 4 - Current best internal CV score: 0.8881878371825942
                                                                              
Generation 5 - Current best internal CV score: 0.8881878371825942
                                                                              
Best pipeline: AdaBoostRegressor(Normalizer(input_matrix, norm=l1), learning_rate=0.01, loss=linear, n_estimators=100)


TPOTRegressor(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=42),
              generations=5, n_jobs=-1, population_size=20, random_state=42,
              scoring='r2', verbosity=2)

In [80]:
# 在测试集上查看R^2分数
score_r2 = tpot.score(X_test, y_test)
print("TPOT model R^2 score: {:.4f}".format(score_r2))

# 如果想要查看其他指标，比如MAE、RMSE等
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = tpot.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("MAE: {:.4f}".format(mae))
print("RMSE: {:.4f}".format(rmse))

TPOT model R^2 score: 0.6521
MAE: 2142248.6949
RMSE: 2642004.8604


In [81]:
# 定义一个RepeatedKFold对象
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# 配置TPOT
tpot = TPOTRegressor(
    generations=10,          # 迭代次数，可根据实际需求增大
    population_size=50,      # 种群大小
    verbosity=2,             # 日志输出级别
    scoring='r2',            # 这里用R^2作为评估指标，也可用MAE/MSE
    random_state=42,         # 结果可复现 
    n_jobs=-1,               # 并行加速，-1表示使用全部CPU
    cv=rkf                   # 将我们定义的RepeatedKFold传给TPOT
)

# 在训练集上进行自动化建模
tpot.fit(X_train, y_train)

is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier


/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/sklearn/base.py:1230: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(
/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/sklearn/base.py:1270: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(


is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
                                                                              
Generation 1 - Current best internal CV score: 0.8301950409128236
                                                                              
Generation 2 - Current best internal CV score: 0.8957464013571023
                                                                              
Generation 3 - Current b


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Library/Fr

                                                                              
Generation 8 - Current best internal CV score: 0.8980734065241407
                                                                              
Generation 9 - Current best internal CV score: 0.9059296522436576
                                                                              
Generation 10 - Current best internal CV score: 0.9059296522436576
                                                                              
Best pipeline: AdaBoostRegressor(SGDRegressor(LinearSVR(LinearSVR(SelectFromModel(SelectFwe(input_matrix, alpha=0.013000000000000001), max_features=0.45, n_estimators=100, threshold=0.35000000000000003), C=0.001, dual=True, epsilon=0.0001, loss=squared_epsilon_insensitive, tol=0.1), C=5.0, dual=False, epsilon=0.1, loss=squared_epsilon_insensitive, tol=0.001), alpha=0.01, eta0=1.0, fit_intercept=True, l1_ratio=0.0, learning_rate=invscaling, loss=epsilon_insensitive, penalty=elastic

TPOTRegressor(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=42),
              generations=10, n_jobs=-1, population_size=50, random_state=42,
              scoring='r2', verbosity=2)

In [82]:
# 在测试集上查看R^2分数
score_r2 = tpot.score(X_test, y_test)
print("TPOT model R^2 score: {:.4f}".format(score_r2))

# 如果想要查看其他指标，比如MAE、RMSE等
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = tpot.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("MAE: {:.4f}".format(mae))
print("RMSE: {:.4f}".format(rmse))

TPOT model R^2 score: 0.7586
MAE: 1714590.5406
RMSE: 2200542.8341


In [76]:
# 定义一个RepeatedKFold对象
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# 配置TPOT
tpot = TPOTRegressor(
    generations=20,          # 迭代次数，可根据实际需求增大
    population_size=100,      # 种群大小
    verbosity=2,             # 日志输出级别
    scoring='r2',            # 这里用R^2作为评估指标，也可用MAE/MSE
    random_state=42,         # 结果可复现 
    n_jobs=-1,               # 并行加速，-1表示使用全部CPU
    cv=rkf                   # 将我们定义的RepeatedKFold传给TPOT
)

# 在训练集上进行自动化建模
tpot.fit(X_train, y_train)

is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor


/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/sklearn/base.py:1230: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(
/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/sklearn/base.py:1270: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(


is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
Optimization Progress:   0%|          | 0/2100 [00:00<?, ?pipeline/s]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Library/Fr

                                                                                
Generation 1 - Current best internal CV score: 0.8889028633043333
                                                                               
Generation 2 - Current best internal CV score: 0.8946457827851162
                                                                               
Generation 3 - Current best internal CV score: 0.8949408371352768
                                                                               
Generation 4 - Current best internal CV score: 0.9027809957775706
                                                                                 
Generation 5 - Current best internal CV score: 0.9027809957775706
Optimization Progress:  30%|██▉       | 626/2100 [11:04<31:19,  1.28s/pipeline]

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Library/Fr

                                                                                
Generation 17 - Current best internal CV score: 0.9069966061481697
                                                                                
Generation 18 - Current best internal CV score: 0.9069966061481697
Optimization Progress:  91%|█████████ | 1901/2100 [39:54<08:26,  2.55s/pipeline]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/inhibitor/Desktop/projects/Doctolib Hackathon/venv/lib/python3.10/site-packages/joblib/externals/loky/backend/popen_loky_posix.py", line 180, in <module>
    exitcode = process_obj._bootstrap()
  File "/Library/Fr

                                                                                
Generation 19 - Current best internal CV score: 0.907924406556765
                                                                                
Generation 20 - Current best internal CV score: 0.907924406556765
                                                                                
Best pipeline: AdaBoostRegressor(ElasticNetCV(RandomForestRegressor(PCA(SelectFwe(input_matrix, alpha=0.027), iterated_power=10, svd_solver=randomized), bootstrap=False, max_features=0.8500000000000001, min_samples_leaf=15, min_samples_split=15, n_estimators=100), l1_ratio=0.8, tol=1e-05), learning_rate=1.0, loss=exponential, n_estimators=100)


TPOTRegressor(cv=RepeatedKFold(n_repeats=3, n_splits=5, random_state=42),
              generations=20, n_jobs=-1, random_state=42, scoring='r2',
              verbosity=2)

In [77]:
# 在测试集上查看R^2分数
score_r2 = tpot.score(X_test, y_test)
print("TPOT model R^2 score: {:.4f}".format(score_r2))

# 如果想要查看其他指标，比如MAE、RMSE等
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = tpot.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("MAE: {:.4f}".format(mae))
print("RMSE: {:.4f}".format(rmse))


TPOT model R^2 score: 0.7520
MAE: 1699399.2583
RMSE: 2230717.3956


In [78]:
model = tpot.fitted_pipeline_

# 用 joblib 保存
import joblib
joblib.dump(model, 'tpot_model.pkl')

# 如果你有 region 的 LabelEncoder，需要一并保存
joblib.dump(label_encoder, 'region_label_encoder.pkl')

['region_label_encoder.pkl']